## Naive Bayes

In this lab, we will implement a Naive Bayes classifier. It is a simple classifier that is often used as a baseline for text classification. It is also a good classifier to use when you have a large number of features.

As the name suggests, the Naive Bayes classifier is based on Bayes' theorem. Bayes' theorem is a way to calculate the probability of an event given some prior knowledge. For example, if we know that a person has a cold, we can calculate the probability that they have a fever. The formula for Bayes' theorem is:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In this case, $A$ is the event that the person has a fever, and $B$ is the event that the person has a cold. $P(A)$, called as the **prior** probability, is the probability of the event before we know anything else. In this case, it is the probability that a person has a fever. $P(B)$ is the probability of the **evidence**, or the probability of the cold. $P(A|B)$ is the probability of the event after we know the evidence. In this case, it is the probability that a person has a fever given that they have a cold. $P(B|A)$ is the probability of the evidence given the event. In this case, it is the probability that a person has a cold given that they have a fever.


The Naive Bayes classifier is based on Bayes' theorem, but it makes a simplifying "Naive" assumption. It assumes that the features are independent of each other. This means that the probability of a feature given the class is the same as the probability of any other feature given the class. For example, if we are classifying emails as spam or not spam, we might use the words "free" and "money" as features. The Naive Bayes classifier assumes that the probability of the word "free" given that the email is spam is the same as the probability of the word "money" given that the email is spam. This is a simplifying assumption, but it is often a good one. In the case of the email example, it is unlikely that the word "free" would appear in a non-spam email, and it is unlikely that the word "money" would appear in a spam email.


Putting this all together, the Naive Bayes classifier calculates the probability of a class given the features using Bayes' theorem. It then chooses the class with the highest probability. The formula for the Naive Bayes classifier is:

$$P(C_k|F_1, F_2, ..., F_n) = \frac{P(F_1, F_2, ..., F_n|C_k)P(C_k)}{P(F_1, F_2, ..., F_n)}$$

In this case, $C_k$ is the class, and $F_1, F_2, ..., F_n$ are the features. $P(C_k)$ is the prior probability of the class. $P(F_1, F_2, ..., F_n)$ is the probability of the evidence, or the probability of the features. $P(F_1, F_2, ..., F_n|C_k)$ is the probability of the evidence given the class. In this case, it is the probability of the features given the class. $P(C_k|F_1, F_2, ..., F_n)$ is the probability of the class given the features. This is the probability that we are trying to calculate. It is the probability that the class is $C_k$ given the features $F_1, F_2, ..., F_n$.

Simplifying the formula, using the chain rule of probability, and assuming that the features are independent, we get:

$$P(C_k|F_1, F_2, ..., F_n) = \frac{P(F_1|C_k)P(F_2|C_k)...P(F_n|C_k)P(C_k)}{P(F_1)P(F_2)...P(F_n)}$$

In this case, we are calculating the probability that the class is $C_k$ given the features $F_1, F_2, ..., F_n$. We do this by multiplying the prior probability of the class by the probability of each feature given the class. We then divide by the probability of the evidence. The probability of the evidence is the same for all classes, so we can ignore it when we are choosing the class with the highest probability. Hence, choosing the class with the highest probability is equivalent to choosing the class with the highest prior probability multiplied by the probability of each feature given the class. It can be expressed as:

$$\hat{C} = \underset{C_k}{\operatorname{argmax}} P(C_k)P(F_1|C_k)P(F_2|C_k)...P(F_n|C_k)$$

In multinomial Naive Bayes, the features are assumed to be counts of the number of times a feature appears in a document. For example, coming back to email spam clasifier, we can count the number of times the words "free" and "money" appear in spam and non-spam emails. We then divide the number of times the word appears in a class by the total number of words in the class. This gives us the probability of the word given the class. Then, we multiply these probabilities together to get the probability of the class given the features. Finally, we need to calculate the prior probability of the class. This is the probability that an email is spam or not spam. We can calculate this by dividing the number of spam emails by the total number of emails.

In this labs, we will use the [20 Newsgroups dataset](http://qwone.com/~jason/20Newsgroups/). This dataset contains 20,000 newsgroup posts, each labeled with one of 20 topics. The dataset is split into 19,000 training posts and 1,000 test posts. The topics are:

```
alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc
```

We will use the training set to train a multinomial Naive Bayes classifier, and then use the test set to evaluate the classifier. We will use the [scikit-learn](http://scikit-learn.org/stable/) library to implement the classifier.

### Loading the data

The data is stored in a directory structure. Each topic is stored in a separate directory. Each post is stored in a separate file. The file name is the post ID. The first line of the file is the post header, which we will ignore. The rest of the file is the post body. The following code loads the data into a list of `(topic, post)` tuples.


In [1]:
import os

def load_data(data_dir):
    data = []
    for topic in os.listdir(data_dir):
        topic_dir = os.path.join(data_dir, topic)
        for fname in os.listdir(topic_dir):
            with open(os.path.join(topic_dir, fname)) as f:
                # Ignore the header
                for line in f:
                    if not line.strip():
                        break

                # Read the rest of the post
                post = f.read()
                data.append((topic, post))
    return data

train_data = load_data('20news-bydate-train')
test_data = load_data('20news-bydate-test')

### Preprocessing

Before we can train the classifier, we need to preprocess the data. We will do the following:

* Convert all words to lowercase
* Remove punctuation
* Remove stop words
* Stem words

The following code defines a function that does all of these preprocessing steps.

In [2]:
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def preprocess(text):
    # Convert words to lower case
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove words that are one character or less
    text = re.sub(r'\b\w{1,2}\b', '', text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([w for w in text.split() if w not in stop_words])

    # Stem words
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(w) for w in text.split()])

    return text

train_data = [(topic, preprocess(post)) for topic, post in train_data]
test_data = [(topic, preprocess(post)) for topic, post in test_data]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mati1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Creating the vocabulary

Now that we have preprocessed the data, we need to create a vocabulary. The vocabulary is a list of all the unique words that appear in the training data. We will use the vocabulary to create a feature vector for each post. The feature vector will contain the frequency of each word in the vocabulary. The following code creates the vocabulary.

In [3]:
from collections import Counter

def create_vocabulary(data, max_words = 10000):
    # Create a list of all the words in the data
    all_words = [word for _, post in data for word in post.split()]

    # Create a dictionary mapping each word to its frequency
    word_counts = Counter(all_words)

    # Sort the words by frequency in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Get the words
    words = [x[0] for x in sorted_word_counts]

    return words[:max_words]


vocabulary = create_vocabulary(train_data)

### Creating the feature vectors

Now that we have the vocabulary, we can create the feature vectors. The feature vector for a post is a list of word frequencies. The $i$th element of the feature vector is the frequency of the $i$-th word in the vocabulary. The following code creates the feature vectors.

In [4]:
def document_features(document):
    document_words = Counter(document.split())
    features = [document_words[word] for word in vocabulary]
    return features

train_features = [(document_features(post), topic) for topic, post in train_data]
test_features = [(document_features(post), topic) for topic, post in test_data]

### Training the classifier

Now that we have the feature vectors, we can train the classifier. We will use the [Naive Bayes classifier](http://scikit-learn.org/stable/modules/naive_bayes.html) from scikit-learn. The following code trains the classifier.

In [5]:
topics = list(set([topic for topic, _ in train_data]))
train_labels = [topics.index(topic) for _, topic in train_features]
test_labels = [topics.index(topic) for _, topic in test_features]

In [6]:
train_samples = [features for features, _ in train_features]
test_samples = [features for features, _ in test_features]

In [7]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(train_samples, train_labels)

MultinomialNB()

### Evaluating the classifier

Now that we have trained the classifier, we can evaluate it on the test set. The following code evaluates the classifier.

In [8]:
from sklearn.metrics import accuracy_score

train_predictions = classifier.predict(train_samples)
test_predictions = classifier.predict(test_samples)

print('Train accuracy:', accuracy_score(train_labels, train_predictions))
print('Test accuracy:', accuracy_score(test_labels, test_predictions))

Train accuracy: 0.9093158918154499
Test accuracy: 0.7635422198619225


In [9]:
from sklearn.metrics import confusion_matrix

print('Topic %-*s Accuracy' % (max([len(topic) for topic in topics]), ''))

cm = confusion_matrix(test_labels, test_predictions)
classes_accuracy = cm.diagonal()/cm.sum(axis=1)
for i, accuracy in enumerate(classes_accuracy):
    print('%-*s %.2f' % (max([len(topic) for topic in topics]) + 6, topics[i], accuracy))

Topic                          Accuracy
alt.atheism                    0.68
soc.religion.christian         0.82
comp.sys.ibm.pc.hardware       0.69
rec.motorcycles                0.91
sci.electronics                0.64
talk.religion.misc             0.43
rec.sport.baseball             0.89
sci.crypt                      0.85
sci.med                        0.81
sci.space                      0.83
talk.politics.guns             0.87
talk.politics.misc             0.55
rec.sport.hockey               0.94
rec.autos                      0.87
talk.politics.mideast          0.81
comp.os.ms-windows.misc        0.55
comp.graphics                  0.73
comp.windows.x                 0.69
comp.sys.mac.hardware          0.78
misc.forsale                   0.72


## Data Science contest #1

Build a spam classifier using the SMS Spam Collection dataset from `spam.csv` file. The dataset contains SMS messages, each labeled as "spam" or "ham" (not spam). Load the data into a dataframe and split it into train and test data set with 20% of the data moved to test set. Preprocess the data, i.e. convert all words to lowercase, remove punctuation, remove stop words, and stem words. Consider using [feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) to create the feature vectors. Train a multinomial Naive Bayes classifier on the training data. Evaluate the classifier on the test data.

In [10]:
import pandas as pd

df = pd.read_csv('spam.csv', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Data science contest #2

Gaussian Naive Bayes is a special case of multinomial Naive Bayes. In Gaussian Naive Bayes, the features are assumed to be continuous values. Specifically, the posterior probability of a class given the features is assumed to be a Gaussian distribution given by:

$$P(C_k|F_1, F_2, ..., F_n) = \frac{1}{\sqrt{2\pi\sigma_k^2}}\exp\left(-\frac{(F_1-\mu_{k1})^2 + (F_2-\mu_{k2})^2 + ... + (F_n-\mu_{kn})^2}{2\sigma_k^2}\right)$$

where $\mu_{ki}$ is the mean of the $i$th feature for class $C_k$, and $\sigma_k$ is the standard deviation of the features for class $C_k$.

Your task is to use the [Iris dataset](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) to train a [Gaussian Naive Bayes classifier](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html). The Iris dataset contains 150 samples of 3 different species of Iris flowers. Each sample has 4 features: sepal length, sepal width, petal length, and petal width. Split the data into train and test sets with 20% of the data moved to the test set. Train a Gaussian Naive Bayes classifier on the training data. Evaluate the classifier on the test data.

In [38]:
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB

iris = load_iris()
X = iris.data
y = iris.target